In [2]:
!pip install earthengine-api
!pip install rasterio
!pip install google-colab
!pip install pyarrow

In [117]:
import ee
import json

ee.Authenticate()
ee.Initialize(project='mse-uhi')

Extracting Image Data (Singapore)

In [ ]:
# Load the Landsat 8 Collection 2 Tier 2 Image Collection
Landsat8_collection = ee.ImageCollection('LANDSAT/LC08/C02/T2_L2')

# Filter by date and location (example date range and bounds for Singapore)
filtered_collection = Landsat8_collection.filterDate('2022-01-01', '2022-12-31').filterBounds(ee.Geometry.Rectangle([103.59, 1.13, 104.07, 1.47]))

# Get the first image from the filtered collection
Landsat8_image = filtered_collection.first()

# If no image is found, handle this condition
if Landsat8_image is None:
    print("No images found for the specified date and region.")
else:
    image_info = Landsat8_image.getInfo()

    # Retrieving band names and other image properties
    band_names = [band['id'] for band in image_info['bands']]
    other_properties = list(image_info['properties'].keys())

    print("Band Names:", band_names)
    print("Other Properties:", other_properties)

    # Select bands
    Landsat8_data = Landsat8_image.select(band_names)

    # Define the region of interest
    singapore = ee.Geometry.Rectangle(103.59, 1.13, 104.07, 1.47)

Band Names: ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL', 'ST_B10', 'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA', 'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT']
Other Properties: ['DATA_SOURCE_ELEVATION', 'WRS_TYPE', 'REFLECTANCE_ADD_BAND_1', 'REFLECTANCE_ADD_BAND_2', 'DATUM', 'REFLECTANCE_ADD_BAND_3', 'REFLECTANCE_ADD_BAND_4', 'REFLECTANCE_ADD_BAND_5', 'REFLECTANCE_ADD_BAND_6', 'REFLECTANCE_ADD_BAND_7', 'system:footprint', 'REFLECTIVE_SAMPLES', 'GROUND_CONTROL_POINTS_VERSION', 'SUN_AZIMUTH', 'DATA_SOURCE_TIRS_STRAY_LIGHT_CORRECTION', 'UTM_ZONE', 'DATE_ACQUIRED', 'ELLIPSOID', 'system:time_end', 'DATA_SOURCE_PRESSURE', 'LANDSAT_PRODUCT_ID', 'STATION_ID', 'TEMPERATURE_ADD_BAND_ST_B10', 'DATA_SOURCE_REANALYSIS', 'REFLECTANCE_MULT_BAND_7', 'system:time_start', 'REFLECTANCE_MULT_BAND_6', 'L1_PROCESSING_LEVEL', 'PROCESSING_SOFTWARE_VERSION', 'L1_DATE_PRODUCT_GENERATED', 'ORIENTATION', 'REFLECTANCE_MULT_BAND_1', 'WRS_ROW', 'REFLECTANCE_MU

In [ ]:
task = ee.batch.Export.image.toDrive(
    image=Landsat8_data.toFloat(),
    description='sg_L8_tier2',
    folder='L8_Data',
    scale=30,
    region=singapore,
    fileFormat='GeoTIFF'
) # check out coordinate system, make sure it's consistent to avoid running into inconsistencies in coordinates/NaN values
task.start()

In [ ]:
print(task.status())

{'state': 'COMPLETED', 'description': 'sg_L8_tier2', 'priority': 100, 'creation_timestamp_ms': 1715698468820, 'update_timestamp_ms': 1715698721400, 'start_timestamp_ms': 1715698475865, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/16sjcvaB9JV3tSH_RQ4t2AOGJ22oGDhMf'], 'attempt': 1, 'batch_eecu_usage_seconds': 18.508819580078125, 'id': 'HNGE44AA5H2QSNQXU6EH6S2H', 'name': 'projects/mse-uhi/operations/HNGE44AA5H2QSNQXU6EH6S2H'}


In [221]:
import rasterio
import pandas as pd
import numpy as np
import os

current_directory = os.getcwd()

file_path = os.path.join(current_directory, 'sg_L8_tier2.tif')

# Open the exported GeoTIFF file.
with rasterio.open(file_path) as src:
    array = src.read()
    nodata_values = src.nodatavals
    band_names = src.descriptions

    # Get the number of bands, image dimensions
    num_bands = src.count
    height, width = src.shape

# Reshape the array so each row corresponds to a pixel and each column to a band
reshaped_array = array.transpose(1, 2, 0).reshape(-1, num_bands)

# Create a DataFrame from the reshaped array with band names
bands_df = pd.DataFrame(reshaped_array, columns=band_names)

# Add pixel coordinates to the DataFrame
bands_df['x'] = np.tile(np.arange(width), height)
bands_df['y'] = np.repeat(np.arange(height), width)

# Scaling and offsetting
bands_df['B10_degC'] = bands_df['ST_B10'] * 0.00341802 + 149 - 273.15

# Replace NoData values in all columns appropriately
for i, nodata_value in enumerate(nodata_values):
    if nodata_value is not None:
        bands_df.iloc[:, i].replace(nodata_value, np.nan, inplace=True)

print(bands_df)

# Saving bands data into a CSV file
csv_path_bands = os.path.join(current_directory, 'Landsat8_Band_Data.csv')
bands_df.to_csv(csv_path_bands, index=False)

           SR_B1    SR_B2    SR_B3    SR_B4    SR_B5    SR_B6    SR_B7  \
0        29886.0  29660.0  28603.0  28251.0  30884.0  22099.0  18719.0   
1        29205.0  29104.0  28086.0  27782.0  30484.0  21758.0  18476.0   
2        28574.0  28517.0  27575.0  27314.0  30066.0  21411.0  18253.0   
3        28204.0  28047.0  27388.0  27036.0  29794.0  21311.0  18195.0   
4        28167.0  27928.0  27628.0  27139.0  29799.0  21499.0  18258.0   
...          ...      ...      ...      ...      ...      ...      ...   
2236405  50127.0  49691.0  47437.0  46772.0  44830.0  15977.0  18241.0   
2236406  50111.0  49707.0  47451.0  46782.0  44855.0  15936.0  18219.0   
2236407  50114.0  49679.0  47474.0  46775.0  44827.0  15943.0  18198.0   
2236408  50148.0  49676.0  47519.0  46811.0  44842.0  15868.0  18135.0   
2236409  50182.0  49729.0  47587.0  46900.0  44927.0  15795.0  18055.0   

         SR_QA_AEROSOL  ST_B10  ST_ATRAN  ...  ST_EMIS  ST_EMSD  ST_QA  \
0                224.0   293.0    317

In [218]:
# Extracting metadata
metadata_dict = Landsat8_image.getInfo()
metadata_json = json.dumps(metadata_dict)

with open('landsat8_metadata.json', 'w') as f:
    f.write(metadata_json)

metadata_df = pd.DataFrame.from_dict(metadata_dict['properties'], orient='index', columns=['value'])
metadata_df = metadata_df.transpose()

print(metadata_df)

# Savings bands data into a CSV  file
csv_path_metadata = os.path.join(current_directory, 'Landsat8_Metadata.csv')
metadata_df.to_csv(csv_path_metadata, index=False)

      DATA_SOURCE_ELEVATION WRS_TYPE REFLECTANCE_ADD_BAND_1  \
value               GLS2000        2                   -0.2   

      REFLECTANCE_ADD_BAND_2  DATUM REFLECTANCE_ADD_BAND_3  \
value                   -0.2  WGS84                   -0.2   

      REFLECTANCE_ADD_BAND_4 REFLECTANCE_ADD_BAND_5 REFLECTANCE_ADD_BAND_6  \
value                   -0.2                   -0.2                   -0.2   

      REFLECTANCE_ADD_BAND_7  ...                     L1_LANDSAT_PRODUCT_ID  \
value                   -0.2  ...  LC08_L1GT_170052_20220802_20220806_02_T2   

      PROCESSING_LEVEL ALGORITHM_SOURCE_SURFACE_REFLECTANCE SPACECRAFT_ID  \
value             L2SP                          LaSRC_1.5.0     LANDSAT_8   

      TEMPERATURE_MINIMUM_BAND_ST_B10 REFLECTIVE_LINES THERMAL_SAMPLES  \
value                      149.003418             7791            7641   

      system:asset_size DATA_SOURCE_AIR_TEMPERATURE          system:index  
value        1051254536                       MODIS 